In [14]:
!pip3 install -r requirements.txt
!pip3 install yfinance --upgrade --no-cache-dir

You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
import pandas as pd
import numpy as np
import yfinance as yf

In [9]:
# session = requests_cache.CachedSession(cache_name='cache', backend='sqlite')

# just add headers to your session and provide it to the reader
# session.headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0', 'Accept': 'application/json;charset=utf-8'}

def get_data_for_multiple_stocks(tickers,start_date,end_date):
    
    stocks = dict()
    # loop through all the tickers
    for ticker in tickers:
        
        # get the data for the specific ticker
        # s = DataReader(ticker, 'stooq', start_date, end_date, session=session)
        s = yf.download(ticker, start_date, end_date)
        #stocks.append(s.rename(mapper={'Adj Close': ticker}, axis=1)[ticker])
        s.insert(0, "Ticker", ticker)  #insert ticker column so you can reference better later
        
        s['Prev Close'] = s['Adj Close'].shift(1)
        s['log_return'] = np.log(s['Adj Close']/s['Prev Close'])
        s['perc_return'] = (s['Adj Close']/s['Prev Close']) - 1
        
        # add it to the dictionary
        stocks[ticker] = s
    # return the dictionary
    #return pd.concat(stocks, axis=1)
    return stocks

In [14]:
# Import dataset 

tickers = ['CL=F']
portfolio =  get_data_for_multiple_stocks(tickers, start_date='2019-01-01',end_date= '2022-12-31')

[*********************100%***********************]  1 of 1 completed


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [16]:
cl_f = portfolio['CL=F'].reset_index()

# Make Date to date type
cl_f['Date'] = cl_f['Date'].apply(str)
cl_f['Date'] = pd.to_datetime(cl_f['Date'], infer_datetime_format=True)

In [17]:
cl_f

,Date,Ticker,Open,High,Low,Close,Adj Close,Volume,Prev Close,log_return,perc_return
0,2019-01-02,CL=F,45.799999,47.779999,44.349998,46.540001,46.540001,850480,NaN,NaN,NaN
1,2019-01-03,CL=F,46.259998,47.490002,45.349998,47.090000,47.090000,788718,46.540001,0.011748,0.011818
2,2019-01-04,CL=F,46.900002,49.220001,46.650002,47.959999,47.959999,817277,47.090000,0.018307,0.018475
3,2019-01-07,CL=F,48.299999,49.790001,48.110001,48.520000,48.520000,819939,47.959999,0.011609,0.011676
4,2019-01-08,CL=F,48.730000,49.950001,48.310001,49.779999,49.779999,765981,48.520000,0.025637,0.025969
...,...,...,...,...,...,...,...,...,...,...,...
1005,2022-12-23,CL=F,78.180000,80.330002,77.980003,79.559998,79.559998,237711,77.489998,0.026363,0.026713
1006,2022-12-27,CL=F,79.900002,81.180000,79.160004,79.529999,79.529999,199601,79.559998,-0.000377,-0.000377
1007,2022-12-28,CL=F,79.889999,79.919998,77.300003,78.959999,78.959999,207717,79.529999,-0.007193,-0.007167
1008,2022-12-29,CL=F,78.820000,78.820000,76.790001,78.400002,78.400002,199858,78.959999,-0.007117,-0.007092
